In [1]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnableSequence, RunnableParallel, RunnablePassthrough, RunnableLambda, RunnableBranch
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os

load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1",
    task = "text-generation",
    huggingfacehub_api_token=os.getenv("HUGINGFACEHUB_API_TOKEN")
)

model = ChatHuggingFace(llm = llm)

parser = StrOutputParser()

prompt1 = PromptTemplate(
    template= "write a detailed report on {topic}",
    input_keys=["topic"]
)

prompt2 = PromptTemplate(
    template= "summarize the following text \n {text}",
    input_keys=["topic"]
)

report_gen_seq_chain = RunnableSequence(prompt1, model, parser)

branched_chain = RunnableBranch(
    (lambda x : len(x.split(" ")) > 500, RunnableSequence(prompt2, model, parser)),
    RunnablePassthrough()
)

final_chain = RunnableSequence(report_gen_seq_chain, branched_chain)

result = final_chain.invoke({"topic" : "dual nature of light"})

print(result)

c:\Users\dheka\OneDrive\Desktop\11.Langchain_from_beginning\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 Light has a dual nature, exhibiting both wave and particle properties. The wave nature of light was demonstrated through experiments like Young's double-slit experiment, diffraction, and polarization, describing light as a transverse electromagnetic wave. The particle nature, or photon theory, was proposed by Einstein to explain the photoelectric effect and Compton scattering, where light behaves as discrete packets of energy. Quantum mechanics reconciles this duality through the wave function, providing a probabilistic description of light's behavior. Light exhibits wave-particle duality, and its properties cannot be precisely determined simultaneously due to the uncertainty principle. This dual nature highlights the limitations of classical physics and the necessity of quantum mechanics for a comprehensive understanding of light and other quantum systems.


In [7]:
final_chain.get_graph().print_ascii()

  +-------------+    
  | PromptInput |    
  +-------------+    
          *          
          *          
          *          
+----------------+   
| PromptTemplate |   
+----------------+   
          *          
          *          
          *          
+-----------------+  
| ChatHuggingFace |  
+-----------------+  
          *          
          *          
          *          
+-----------------+  
| StrOutputParser |  
+-----------------+  
          *          
          *          
          *          
    +--------+       
    | Branch |       
    +--------+       
          *          
          *          
          *          
  +--------------+   
  | BranchOutput |   
  +--------------+   
